# Face Sonification Prototype

ISSUES:
* if you launch a synth (s_new), it's going to take some time until it is up and running: instantiation of a synth is asynchronous
    * we may lose some messages before the synth is ready to play
    * we should:
        * start the synths in advance (maybe paused)
        * pause synths when they are not useful instead of stopping them
* often, we have a video recorded with a frame rate and sensor data that has been recorded with another frame rate
    * PyQtGraph
    * schedule visual events in the time queue
* augmentation of the video with custom plotting, e.g. yourself plotting markers instead of having them plotted by OpenFace
    * discuss if and how

* The timing problem that OpenFace has seems to be related with OpenCV. The tracked video sometimes is sligtly shorter than the original one and sometimes it is longer. It seems that OpenCV has some problems working with some of the codecs.
    * ffmpeg -i video.file -r 30 -vcodec ffv1 -acodec pcm_s16le output_name
        * in my case, this does not work
    * ffmpeg -i video.file -r 30 -vcodec ffv1 output_name
    * split video frames into images if the previous solution fails.
        * ffmpeg

* We can consider the idea of focusing on the AUs that OpenFace detects with good precision and maybe cut off some of the others. We can find this information in the OpenFace paper.
* The smile seems to be in many context the most important source of information. We can consider the idea of designing sonifications to put this element in the foreground.
* The event-based sonification for how I implemented can be confusing for the listener: events are triggered only when the intensity of the AU goes over some thresholds (i.e. 1, 2, 3, 4). This way, the events are able to provide some information on the dinamic behaviour of the sound, but it also means that if an expression stays in the same range for a long time, we don't really have a continuous feedback, as the sound fades out quickly. Another problem is that events are also triggered when the intensity pass from a high range to a lower range; even if this provides some feedback on the dynamic behaviour, it is counterintuitive, because the user would expect the event to be associated to the activation of an expression.
* We can try to use envelope parameters to handle the fall down
* We can try a more hybrid approach. Some AUs will be sonified using an event-based approach that will omit information relative to the intensity, so that one event for expression will be triggered, e.g. one blink is sonified as one drop: we ignore the information on the intensity of the blink; we can do this for many other expression that are not our primary focus. The most important expressions (e.g. the smile), will be sonified somehow continuously, so that the listener would be able to perceive their stationary behaviour as well.

* We could think of a musical sonification when AUs are associated to tracks that would play well together. When an AU is present, the relative track is played. AUs that are incompatible can be mapped to tracks that don's sound well together.
    * Rithmical structured tracks
    * Ambient
* Sonify the movements of the head and of the eyes. We can sonify the degree of variation to the standard direction, so that when e.g. the head is in a standard position, no sound is played.

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

import panson as ps
from panson import bundle

## OpenFace workaround
It works, but...
* we have a very long conversion phase
* the feature extraction is much longer

In [ ]:
!mkdir full
!ffmpeg -i full.avi -f image2 full/video-frame%05d.jpg

In [ ]:
!docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -fdir files/full -aus -gaze -pose -tracked

In [ ]:
# 0 is the maximum quality
!ffmpeg -y -i full.avi -vcodec mpeg4 -q:v 0 -max_muxing_queue_size 2048 full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset slow -crf 22 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset slow -crf 0 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset veryslow -crf 0 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec mpeg4 -q:v 1 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

## Container setup and video processing

The following section assumes that OpenFace was installed using docker.

As first step, run the container with the following command:
* `docker run -it --rm --name openface --mount type=bind,source=/home/michele/Desktop/Thesis/media/files,target=/home/openface-build/files algebr/openface:latest`
* Substitute /home/michele/Desktop/Thesis/media/files with the mounting point
* The current working directory must contain a directory **files/**
    * This directory is bound with the --mount option to the files/ directory present in the docker container
    * This directory is shared between the file system of the host and the one of the container

Now we can launch the OpenFace executables (present in the container) from outside the container environment with a command similar to the following:
* `docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -f files/video.avi`
* From python (this notebook) we can call the same process and read the results
* Create the directory files/processed in advance

In [ ]:
!xhost +

In [ ]:
# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')

def feature_extraction_offline(video_name):
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

def feature_extraction_online(pipe='files/pipe'):
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def kill_feature_extraction_online():
    # !docker exec -it openface pkill FeatureExt
    command = ['docker', 'exec', CONTAINER_NAME, 'pkill', 'FeatureExt']
    subprocess.run(command)

def read_openface_csv(csv_path):
    return pd.read_csv(csv_path, sep=r',\s*', engine='python')

In [ ]:
def ffmpeg_convert(in_file, out_file):
    
    # this was the original command that appeared in the notebook
    # ffmpeg -y -i "files/processed/phone.avi" -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 128k "files/phone-processed.mp4"
    command = [
        'ffmpeg', '-y',
        '-i', in_file,
        '-c:v', 'libx264',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-c:a', 'libvo_aacenc',
        '-b:a', '128k',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

def ffmpeg_merge(video_file, audio_file, out_file):
    
    # ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y
                
    command = [
        'ffmpeg', '-y',
        '-i', video_file,
        '-i', audio_file,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

In [ ]:
video_name = 'full-rigth-codecs.avi'

In [ ]:
%time feature_extraction_offline(video_name)

In [ ]:
ffmpeg_convert("files/processed/phone.avi", "files/phone-processed.mp4")

## Sonifications

### Setup

Here we are using the current **develop** branch of sc3nb.

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

If this does not work, open QJackCtl and link the nodes in the graph manually.

In [ ]:
sc.exit()

In [ ]:
# test sound output
sc.server.blip()

In [ ]:
sc.server.latency = 0.1

### Sonification 1: AU04 Test
This is a very simple sonification of AU04 (Brow Lowerer). The intensity of AU04 is used here to modulate both the amplitude and the frequency of a continuous synth. As continuous synth, the default synth of sc3nb s2 is used (we will have to instruct the server to load it).

* The intensity range \[0,1\] is mapped into the amplitude range \[0,0.3\], where 0.3 will be the maximum amplitude of the sound. The sonification has a parameter amp that can be used to scale this range.
* The intensity range \[0,5\] is mapped into the midi range \[69,81\]

In [ ]:
# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.SliderParameter(0, 1, 0.01)
    
    def __init__(self):
        super().__init__()
        self.amp = 1
    
    @bundle
    def _initialize(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def _start(self):
        self.synth = scn.Synth("s2", {"amp": 0})
    
    @bundle
    def _stop(self):
        self._s.free_all()
    
    @bundle
    def _process(self, row):
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
son = AU04ContinuousSonification()
son

#### Offline

In [ ]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU04_r', 'AU12_r'], queue_size=50)
dp = ps.DataPlayer(son, feature_display=feature_display).load(df)

In [ ]:
feature_display.show(fps=30)
display(son)
display(dp)

In [ ]:
dp.rate = 1

In [ ]:
dp.export("score.wav")

#### Online

In [ ]:
import csv

FIFO = os.path.join(FILE_DIR, 'pipe.csv')

def data_generator():
    with open(FIFO, 'r') as fifo:
        # the reader attempts to execute fifo.readline()
        # which blocks if there are no lines
        reader = csv.reader(fifo, skipinitialspace=True)
        
        header = next(reader)
        
        # the loop ends when the pipe is closed from the writing side
        for i, row in enumerate(reader):
            series = pd.Series(row, header, dtype='float', name=i)
            yield series

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU04_r', 'AU12_r'], 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(fps=20)
display(son)
rtdp

### Sonification 2: Pentatonic eyes + brows
* 1: Inner Brow Raiser
* 2: Outer Brow Raiser (unilateral)
* 4: Brow Lowerer
* 5: Upper Lid Raiser
* 6: Cheek Raiser
* 7: Lid Tightener

In [ ]:
class PentatonicContinuous(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.SliderParameter(0, 1, 0.01)
    
    BASE_TONE = 69
    PENTATONIC = [
        scn.midicps(BASE_TONE),
        scn.midicps(BASE_TONE + 3),
        scn.midicps(BASE_TONE + 5),
        scn.midicps(BASE_TONE + 7),
        scn.midicps(BASE_TONE + 10),
        scn.midicps(BASE_TONE + 12)
    ]
    
    def __init__(self):
        super().__init__()
        
        # parameters default
        self.amp = 1
    
    @bundle
    def _initialize(self):
        # load default synth s2
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def _start(self):
        self.au01_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[5]})
        self.au02_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[4]})
        self.au04_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[3]})
        self.au05_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[2]})
        self.au06_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[0]})
        self.au07_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[1]})
    
    @bundle
    def _stop(self):
        self._s.free_all()
    
    @bundle
    def _process(self, row):
        intensity = row["AU01_r"]
        amp = self.map_intensity(intensity)
        self.au01_synth.set("amp", self.amp * amp)
        
        intensity = row["AU02_r"]
        amp = self.map_intensity(intensity)
        self.au02_synth.set("amp", self.amp * amp)
        
        intensity = row["AU04_r"]
        amp = self.map_intensity(intensity)
        self.au04_synth.set("amp", self.amp * amp)
        
        intensity = row["AU05_r"]
        amp = self.map_intensity(intensity)
        self.au05_synth.set("amp", self.amp * amp)
        
        intensity = row["AU06_r"]
        amp = self.map_intensity(intensity)
        self.au06_synth.set("amp", self.amp * amp)
        
        intensity = row["AU07_r"]
        amp = self.map_intensity(intensity)
        self.au07_synth.set("amp", self.amp * amp)
        
    @staticmethod
    def map_intensity(intensity):
        if intensity < 1.0:
            amp = 0
        else:
            db = scn.linlin(intensity, 1, 5, -20, -5, "minmax")
            amp = scn.dbamp(db)

        return amp

In [ ]:
son = PentatonicContinuous()
son

In [ ]:
interesting_features = ["AU01_r", "AU02_r", "AU04_r", "AU05_r", "AU06_r", "AU07_r"]
feature_display = ps.LiveFeatureDisplay(interesting_features, 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(fps=10)
display(son)
display(rtdp)

### Sonification 3: Pentatonic eyes + brows with MdaPiano

In [ ]:
class PentatonicMda(ps.Sonification):
    
    BASE_TONE = 69
    PENTATONIC = [
        scn.midicps(BASE_TONE),
        scn.midicps(BASE_TONE + 3),
        scn.midicps(BASE_TONE + 5),
        scn.midicps(BASE_TONE + 7),
        scn.midicps(BASE_TONE + 10),
        scn.midicps(BASE_TONE + 12)
    ]
    
    piano_def = scn.SynthDef(
        "mdapiano",
        r"""{ |freq=440, gate=1, vel=100, amp=1|
            var piano = MdaPiano.ar(
                freq,
                gate,
                vel,
                decay: 0,
                release: 0,
                hard: 0,
                stereo: 0,
                sustain: 0,
                mul: amp
            );
            DetectSilence.ar(piano, 0.01, doneAction:2);
            Out.ar(0, piano);
        }"""
    )
    
    @bundle
    def _initialize(self):
        self.piano_def.add()

    @bundle
    def _start(self):
        self.old_range_level_01 = 0
        self.old_range_level_02 = 0
        self.old_range_level_04 = 0
        self.old_range_level_05 = 0
        self.old_range_level_06 = 0
        self.old_range_level_07 = 0
        self.old_range_level_09 = 0
        self.old_range_level_10 = 0
        self.old_range_level_12 = 0
        self.old_range_level_14 = 0
        self.old_range_level_15 = 0
        self.old_range_level_17 = 0
        self.old_range_level_20 = 0
        self.old_range_level_23 = 0
        self.old_range_level_25 = 0
        self.old_range_level_26 = 0
    
    @bundle
    def _stop(self):
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU01_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_01, cur_range_level, self.PENTATONIC[5])
        self.old_range_level_01 = cur_range_level
        
        intensity = row["AU02_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_02, cur_range_level, self.PENTATONIC[4])
        self.old_range_level_02 = cur_range_level
        
        intensity = row["AU04_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_04, cur_range_level, self.PENTATONIC[3])
        self.old_range_level_04 = cur_range_level
        
        intensity = row["AU05_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_05, cur_range_level, self.PENTATONIC[2])
        self.old_range_level_05 = cur_range_level
        
        intensity = row["AU06_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_06, cur_range_level, self.PENTATONIC[0])
        self.old_range_level_06 = cur_range_level
        
        intensity = row["AU07_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_07, cur_range_level, self.PENTATONIC[1])
        self.old_range_level_07 = cur_range_level        
        
    @staticmethod
    def map_range(old_range_level, cur_range_level, freq):
        if cur_range_level != old_range_level and cur_range_level >= 1:
            vel = scn.linlin(cur_range_level, 1, 5, 40, 127)
            scn.Synth('mdapiano', {"freq": freq, "vel": vel})

In [ ]:
son = PentatonicMda()
son

TODO: adjust amplitude

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
rtdp

### Sonification 4: Drop blink

In [ ]:
class BlinkDrop(ps.Sonification):
    
    # drop definition
    drop_def = scn.SynthDef(
        "drop",
        r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
            var sig, env, fch;
            fch = XLine.kr(freq, freq+dp, dur);
            sig = SinOsc.ar(fch);
            env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
            Out.ar(0, Pan2.ar(sig, pan, env*amp))
        }"""
    )
    
    @bundle
    def _initialize(self):
        self.drop_def.add()

    @bundle
    def _start(self):
        self.blinking = False
    
    @bundle
    def _stop(self):
        # drops die out alone
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < 1:
                self.blinking = False
        elif intensity >= 1:
            self.blinking = True
            scn.Synth("drop")

In [ ]:
son = BlinkDrop()
son

In [ ]:
feature_display = ps.LiveFeatureDisplay(["AU45_r"], 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(20)
rtdp

### Sonification 5: Multi-percussion sound (event-based)

Similar samples
* snares
<!--     * <audio controls src="samples/132417__sajmund__percussion-clave-like-hit.wav"/> -->
<!--     * <audio controls src="samples/277397__earmark-audio__efev1-percussion-snare.wav"/> -->
* high bells
<!--     * <audio controls src="samples/339808__inspectorj__hand-bells-c-db-single.wav"/> -->
<!--     * <audio controls src="samples/360327__inspectorj__triangle-8-hard-hit-a.wav"/> -->
<!--     * <audio controls src="samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"/> -->
<!--     * <audio controls src="samples/387715__jagadamba__gong-percussion.wav"/> -->
<!--     * <audio controls src="samples/414563__pjcohen__agogo-bell-low-velocity11.wav"/> -->
<!--     * <audio controls src="samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"/> -->
<!--     * <audio controls src="samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"/> -->
* crash
<!--     * <audio controls src="samples/public domain/209874__veiler__pff-chrash-14.wav"/> -->
* electronic
<!--     * <audio controls src="samples/public domain/487662__phonosupf__electronic-percussion-5.wav"/> -->
<!--     * <audio controls src="samples/207919__altemark__space-snare.wav"/> -->
<!--     * <audio controls src="samples/577014__nezuai__cartoon-percussion-3.wav"/> -->
* drums
<!--     * <audio controls src="samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"/> -->
    * <audio controls src="samples/public domain/439828__twentytwentymusic__electronic-percussion-6.wav"/>
<!--     * <audio controls src="samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"/> -->
* bass drum
<!--     * <audio controls src="samples/138358__minorr__bass-drum-p.wav"/> -->
<!--     * <audio controls src="samples/234746__sonidotv__legno-10.wav"/> -->

Concepts:
* Sounds that usually plays together should be distinguishable
* Sounds that play more often should be less intrusive
* One area should have sounds that are somehow related

<hr>

* 1: Inner Brow Raiser - <audio controls src="samples/360327__inspectorj__triangle-8-hard-hit-a.wav"/>
* 2: Outer Brow Raiser (unilateral) - <audio controls src="samples/339808__inspectorj__hand-bells-c-db-single.wav"/>
* 4: Brow Lowerer - <audio controls src="samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"/>

<hr>

* 5: Upper Lid Raiser - <audio controls src="samples/277397__earmark-audio__efev1-percussion-snare.wav"/>
* 6: Cheek Raiser - <audio controls src="samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"/>
* 7: Lid Tightener - <audio controls src="samples/132417__sajmund__percussion-clave-like-hit.wav"/>

<hr>

* 9: Nose Wrinkler (usually goes along with 4 and 10) - <audio controls src="samples/577014__nezuai__cartoon-percussion-3.wav"/>
* 10: Upper Lip Raiser - <audio controls src="samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"/>

<hr>

* 12: Lip Corner Puller - <audio controls src="samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"/>
* 14: Dimpler - <audio controls src="samples/public domain/487662__phonosupf__electronic-percussion-5.wav"/>
* 15: Lip Corner Depressor - <audio controls src="samples/387715__jagadamba__gong-percussion.wav"/>
* 20: Lip Stretcher - <audio controls src="samples/414563__pjcohen__agogo-bell-low-velocity11.wav"/>

<hr>

* 23: Lip Tightener - <audio controls src="samples/public domain/209874__veiler__pff-chrash-14.wav"/>

<hr>

* 17: Chin Raiser - <audio controls src="samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"/>
* 25: Lips Part (relax Mentalis, antagonist of AU17) - <audio controls src="samples/138358__minorr__bass-drum-p.wav"/>
* 26: Jaw Drop (usually goes along with 25) - <audio controls src="samples/234746__sonidotv__legno-10.wav"/>

<hr>

* 28: Lip Suck (usually along with 26) - <audio controls src="samples/207919__altemark__space-snare.wav"/>
    * OpenFace only provides presence information

<hr>

* 45: Blink (drop)

In [ ]:
AU01_SAMPLE_PATH = "samples/360327__inspectorj__triangle-8-hard-hit-a.wav"
AU02_SAMPLE_PATH = "samples/339808__inspectorj__hand-bells-c-db-single.wav"
AU04_SAMPLE_PATH = "samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"
AU05_SAMPLE_PATH = "samples/277397__earmark-audio__efev1-percussion-snare.wav"
AU06_SAMPLE_PATH = "samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"
AU07_SAMPLE_PATH = "samples/132417__sajmund__percussion-clave-like-hit.wav"
AU09_SAMPLE_PATH = "samples/577014__nezuai__cartoon-percussion-3.wav"
AU10_SAMPLE_PATH = "samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"
AU12_SAMPLE_PATH = "samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"
AU14_SAMPLE_PATH = "samples/public domain/487662__phonosupf__electronic-percussion-5.wav"
AU15_SAMPLE_PATH = "samples/387715__jagadamba__gong-percussion.wav"
AU17_SAMPLE_PATH = "samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"
AU20_SAMPLE_PATH = "samples/414563__pjcohen__agogo-bell-low-velocity11.wav"
AU23_SAMPLE_PATH = "samples/public domain/209874__veiler__pff-chrash-14.wav"
AU25_SAMPLE_PATH = "samples/138358__minorr__bass-drum-p.wav"
AU26_SAMPLE_PATH = "samples/234746__sonidotv__legno-10.wav"
AU28_SAMPLE_PATH = "samples/207919__altemark__space-snare.wav"

In [ ]:
class Percussions(ps.Sonification):
    
    drop_def = scn.SynthDef(
        "drop",
        r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
            var sig, env, fch;
            fch = XLine.kr(freq, freq+dp, dur);
            sig = SinOsc.ar(fch);
            env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
            Out.ar(0, Pan2.ar(sig, pan, env*amp))
        }"""
    )

    # define custom playbuf synth that frees the synth when the buffer fades out
    playbuf1_def = scn.SynthDef(
        "playbuf1",
        r"""{| out=0, bufnum=0, rate=1, amp=0.2, pan=0 |
            var sig;
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            DetectSilence.ar(sig, 0.1, doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )

    # Number of channels that the buffer will be. This must be a fixed integer. The architecture of the SynthDef cannot change after it is compiled.
    playbuf2_def = scn.SynthDef(
        "playbuf2",
        r"""{| out=0, bufnum=0, rate=1, amp=0.2, pan=0 |
            var sig;
            sig = PlayBuf.ar(2, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            DetectSilence.ar(sig, 0.1, doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )

    @bundle
    def _initialize(self):
        self.drop_def.add()
        self.playbuf1_def.add()
        self.playbuf2_def.add()
        
        self.buf01 = scn.Buffer().read(AU01_SAMPLE_PATH)
        self.buf02 = scn.Buffer().read(AU02_SAMPLE_PATH)
        self.buf04 = scn.Buffer().read(AU04_SAMPLE_PATH)
        self.buf05 = scn.Buffer().read(AU05_SAMPLE_PATH)
        self.buf06 = scn.Buffer().read(AU06_SAMPLE_PATH)
        self.buf07 = scn.Buffer().read(AU07_SAMPLE_PATH)
        self.buf09 = scn.Buffer().read(AU09_SAMPLE_PATH)
        self.buf10 = scn.Buffer().read(AU10_SAMPLE_PATH)
        self.buf12 = scn.Buffer().read(AU12_SAMPLE_PATH)
        self.buf14 = scn.Buffer().read(AU14_SAMPLE_PATH)
        self.buf15 = scn.Buffer().read(AU15_SAMPLE_PATH)
        self.buf17 = scn.Buffer().read(AU17_SAMPLE_PATH)
        self.buf20 = scn.Buffer().read(AU20_SAMPLE_PATH)
        self.buf23 = scn.Buffer().read(AU23_SAMPLE_PATH)
        self.buf25 = scn.Buffer().read(AU25_SAMPLE_PATH)
        self.buf26 = scn.Buffer().read(AU26_SAMPLE_PATH)
        # self.buf28 = scn.Buffer().read(AU28_SAMPLE_PATH)

    @bundle
    def _start(self):
        self.blinking = False
        
        self.old_range_level_01 = 0
        self.old_range_level_02 = 0
        self.old_range_level_04 = 0
        self.old_range_level_05 = 0
        self.old_range_level_06 = 0
        self.old_range_level_07 = 0
        self.old_range_level_09 = 0
        self.old_range_level_10 = 0
        self.old_range_level_12 = 0
        self.old_range_level_14 = 0
        self.old_range_level_15 = 0
        self.old_range_level_17 = 0
        self.old_range_level_20 = 0
        self.old_range_level_23 = 0
        self.old_range_level_25 = 0
        self.old_range_level_26 = 0
    
    @bundle
    def _stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        # cast intensity to integer
        cur_range_level_01 = int(row["AU01_r"])
        self.map_range(self.old_range_level_01, cur_range_level_01, self.buf01)
        # update old_range_level
        self.old_range_level_01 = cur_range_level_01
        
        cur_range_level_02 = int(row["AU02_r"])
        self.map_range(self.old_range_level_02, cur_range_level_02, self.buf02)
        self.old_range_level_02 = cur_range_level_02
        
        cur_range_level_04 = int(row["AU04_r"])
        self.map_range(self.old_range_level_04, cur_range_level_04, self.buf04)
        self.old_range_level_04 = cur_range_level_04
        
        cur_range_level_05 = int(row["AU05_r"])
        self.map_range(self.old_range_level_05, cur_range_level_05, self.buf05)
        self.old_range_level_05 = cur_range_level_05
        
        cur_range_level_06 = int(row["AU06_r"])
        self.map_range(self.old_range_level_06, cur_range_level_06, self.buf06)
        self.old_range_level_06 = cur_range_level_06
        
        cur_range_level_07 = int(row["AU07_r"])
        self.map_range(self.old_range_level_07, cur_range_level_07, self.buf07)
        self.old_range_level_07 = cur_range_level_07
        
        cur_range_level_09 = int(row["AU09_r"])
        self.map_range(self.old_range_level_09, cur_range_level_09, self.buf09)
        self.old_range_level_09 = cur_range_level_09
        
        cur_range_level_10 = int(row["AU10_r"])
        self.map_range(self.old_range_level_10, cur_range_level_10, self.buf10)
        self.old_range_level_10 = cur_range_level_10
        
        cur_range_level_12 = int(row["AU12_r"])
        self.map_range(self.old_range_level_12, cur_range_level_12, self.buf12)
        self.old_range_level_12 = cur_range_level_12
        
        cur_range_level_14 = int(row["AU14_r"])
        self.map_range(self.old_range_level_14, cur_range_level_14, self.buf14)
        self.old_range_level_14 = cur_range_level_14
        
        cur_range_level_15 = int(row["AU15_r"])
        self.map_range(self.old_range_level_15, cur_range_level_15, self.buf15)
        self.old_range_level_15 = cur_range_level_15
        
        cur_range_level_17 = int(row["AU17_r"])
        self.map_range(self.old_range_level_17, cur_range_level_17, self.buf17)
        self.old_range_level_17 = cur_range_level_17        
        
        cur_range_level_20 = int(row["AU20_r"])
        self.map_range(self.old_range_level_20, cur_range_level_20, self.buf20)
        self.old_range_level_20 = cur_range_level_20
        
        cur_range_level_23 = int(row["AU23_r"])
        self.map_range(self.old_range_level_23, cur_range_level_23, self.buf23)
        self.old_range_level_23 = cur_range_level_23
        
        cur_range_level_25 = int(row["AU25_r"])
        self.map_range(self.old_range_level_25, cur_range_level_25, self.buf25)
        self.old_range_level_25 = cur_range_level_25
        
        cur_range_level_26 = int(row["AU26_r"])
        self.map_range(self.old_range_level_26, cur_range_level_26, self.buf26)
        self.old_range_level_26 = cur_range_level_26
        
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < 1:
                self.blinking = False
        elif intensity >= 1:
            self.blinking = True
            scn.Synth("drop")
    
    @staticmethod
    def map_range(old_range_level, cur_range_level, buf):

        if cur_range_level != old_range_level and cur_range_level >= 1:
            db = scn.linlin(cur_range_level, 1, 5, -20, 0, "minmax")
            amp = scn.dbamp(db)

            if buf.channels == 1:
                synth = "playbuf1"
            else:
                synth = "playbuf2"

            scn.Synth(synth, {"bufnum": buf.bufnum, "amp": amp})

In [ ]:
son = Percussions()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
rtdp